## 不均衡データへの対応

In [1]:
!git clone https://github.com/saiku122/AIJobcolle.git

Cloning into 'AIJobcolle'...
remote: Enumerating objects: 279, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 279 (delta 108), reused 155 (delta 46), pack-reused 0
Receiving objects: 100% (279/279), 11.70 MiB | 12.76 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [2]:
cd /content/AIJobcolle/MachineLearning/python

/content/AIJobcolle/MachineLearning/python


まず分類用のサンプルデータであるローン審査データを読み込みます。<br>データ前処理はone-hotエンコーディングと欠損値補完までを行っています。

In [5]:
import pandas as pd
from sklearn.impute import SimpleImputer

# import data
df = pd.read_csv('./data/av_loan_u6lujuX_CVtuZ9i.csv', header=0)
X = df.iloc[:, :-1]          # 最終列外を特徴量X
X = X.drop('Loan_ID',axis=1) # Loan_IDはID情報のため特徴量から削除
y = df.iloc[:,-1]            # 最終列を正解データ

# ローン審査でNOとなったサンプルを1に変換
class_mapping = {'N':1, 'Y':0}
y = y.map(class_mapping)

In [6]:
# one-hot エンコーディング
ohe_columns = ['Dependents',
               'Gender',
               'Married',
               'Education',
               'Self_Employed',
               'Property_Area']

X_ohe = pd.get_dummies(X,
                       dummy_na=True,
                       columns=ohe_columns)

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X_ohe)
X_ohe_columns = X_ohe.columns.values
X_ohe = pd.DataFrame(imp.transform(X_ohe), columns=X_ohe_columns)

display(X_ohe.head())

NameError: ignored

### アンダーサンプリング

アンダーサンプリングの実装例です。<br>負例（422件）が正例の件数（192件）まで削減されていることが確認できます。

In [ ]:
# ランダムアンダーサンプリング
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_under, y_under = rus.fit_sample(X_ohe, y)
Counter(y_under)

### オーバーサンプリング

以下、オーバーサンプリングの実装例です。<br>正例（192件）が負例の件数（422件）まで増加していることが確認できます。

In [ ]:
# ランダムオーバーサンプリング, SMOTE
from imblearn.over_sampling import RandomOverSampler,SMOTE
ros = RandomOverSampler(random_state=0)
smt = SMOTE(random_state=0)

X_over,y_over = ros.fit_sample(X_ohe, y)
X_smt,y_smt = smt.fit_sample(X_ohe, y)

print('Random Over Sampler',Counter(y_over))
print('SMOTE', Counter(y_smt))

最後に、不均衡対応別のモデルの比較評価の実行例を確認します。<b><br>モデルの評価用データはリサンプリング前に確保されるべきで点に留意しましょう。</b>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,f1_score

# holdout
X_train,X_test,y_train,y_test= train_test_split(X_ohe,
                                                y,
                                                test_size=0.20,
                                                random_state=0)
# resampling
X_train_under, y_train_under = rus.fit_sample(X_train, y_train)
X_train_over, y_train_over = ros.fit_sample(X_train, y_train)
X_train_smt, y_train_smt = smt.fit_sample(X_train, y_train)

# modeling
pipe_gb = Pipeline([('scl',StandardScaler()),
                    ('est',GradientBoostingClassifier(random_state=1))])
# evaluation
###############################################
pipe_gb.fit(X_train,
            y_train)
print('Original Train:', 
      f1_score(y_train,
               pipe_gb.predict(X_train)))
print('Original Test:', 
      f1_score(y_test,
               pipe_gb.predict(X_test)))
###############################################
pipe_gb.fit(X_train_under,
            y_train_under)
print('Undersampling Train:',
      f1_score(y_train_under,
               pipe_gb.predict(X_train_under)))
print('Undersampling Test:', 
      f1_score(y_test,
               pipe_gb.predict(X_test)))
###############################################
pipe_gb.fit(X_train_over,
            y_train_over)
print('Oversampling Train:',
      f1_score(y_train_over,
               pipe_gb.predict(X_train_over)))
print('Oversampling Test:',
      f1_score(y_test,
               pipe_gb.predict(X_test)))
###############################################
pipe_gb.fit(X_train_smt,
            y_train_smt)
print('SMOTE Train:',
      f1_score(y_train_smt,
               pipe_gb.predict(X_train_smt)))
print('SMOTE Test:',
      f1_score(y_test,
               pipe_gb.predict(X_test)))

上記結果から、本データについては不均衡を「完全に解消する」メリットは見られませんでした。不均衡を解消する前のモデルパフォーマンスを定量的に把握した上で、不均衡をどの程度解消するとパフォーマンスがどのように変化するかを記録し、エビデンスを持った対応を心掛けましょう。